In [24]:
from utils import *
encoder = 'scibert'
pooling = 'mean'
path = r'./Data/data02.tsv'
Data = DataCenter(path)
division_file = r'./Data/index5.tsv'
division_id = Sampling(division_file)
print(division_id.keys())
TrainSet = Dataset(Data,'train_set',division_id['train_idx:'])
ValidSet = Dataset(Data,'valid_set',division_id['valid_idx:'])
TestSet = Dataset(Data,'test_set',division_id['test_idx:'])

data loaded
Dataset loaded ......
dict_keys(['train_idx:', 'test_idx:', 'valid_idx:'])


In [25]:
import torch
from LanguageModel import *
device = torch.device('cpu')
LM = WordEmbedding(encoder,device)

seq_len,input_ids, word_span,attention_mask = LM.Tokenize(TrainSet.Sentence_List)
train_embedding = LM.Embedding(input_ids,attention_mask)
train_embedding = train_embedding.detach()
print(train_embedding.size())

v_seq_len,v_input_ids, v_word_span,v_attention_mask = LM.Tokenize(ValidSet.Sentence_List)
v_embedding = LM.Embedding(v_input_ids,v_attention_mask)
v_embedding = v_embedding.detach()
print(v_embedding.size())

t_seq_len,t_input_ids,t_word_span,t_attention_mask = LM.Tokenize(TestSet.Sentence_List)
t_embedding = LM.Embedding(t_input_ids,t_attention_mask)
t_embedding = t_embedding.detach()
print(t_embedding.size())

loading scibert ......
torch.Size([592, 122, 768])
torch.Size([127, 101, 768])
torch.Size([126, 100, 768])


In [26]:
test_emb,train_sen,train_fc = Embedding_Extraction(TestSet,t_word_span,t_embedding,pooling,device)
train_emb,test_sen,test_fc = Embedding_Extraction(TrainSet,word_span,train_embedding,pooling,device)
valid_emb,valid_sen,valid_fc = Embedding_Extraction(ValidSet,v_word_span,v_embedding,pooling,device)


print(train_emb.size())
print(valid_emb.size())
print(test_emb.size())

torch.Size([2790, 3, 1536])
torch.Size([757, 3, 1536])
torch.Size([713, 3, 1536])


**Unsupvised Cosine**

In [27]:
def Unsupvised(pred,truth:list):
    shredhold = 0
    best_Accuracy = 0
    for v in pred:
        A,P,R,F1 = Evaluatation(pred,truth,v)
        if A > best_Accuracy:
            shredhold = v
            best_Accuracy = A
        else:pass
    return shredhold

def Find_shredhold(pred:list,truth:list):
    shredhold = 0
    best_Accuracy,best_p,best_r,best_F1 = 0,0,0,0
    l = len(truth)
    for i in range(l):
        A,P,R,F1 = Evaluatation(pred,truth,pred[i])
        if A > best_Accuracy:
            shredhold = pred[i]
            best_Accuracy,best_p,best_r,best_F1 = A,P,R,F1
        else:pass
    return shredhold,best_Accuracy,best_p,best_r,best_F1
shredhold_span,a_s,p_s,r_s,F1_s = Find_shredhold(np.array(torch.cosine_similarity(train_emb[:,:,:LM.dim],train_emb[:,:,LM.dim:],dim=-1)[:,0]),TrainSet.SR_List)
print(shredhold_span)
Ta_s,Tp_s,Tr_s,TF1_s = Evaluatation(np.array(torch.cosine_similarity(test_emb[:,:,:LM.dim],test_emb[:,:,LM.dim:],dim=-1)[:,0]),TestSet.SR_List,shredhold_span)
Va_s,Vp_s,Vr_s,VF1_s = Evaluatation(np.array(torch.cosine_similarity(valid_emb[:,:,:LM.dim],valid_emb[:,:,LM.dim:],dim=-1)[:,0]),ValidSet.SR_List,shredhold_span)
print('Accuary:%.2f\tPrecision:%.2f\tRecall:%.2f\tF1:%.2f\n'%(Va_s,Vp_s,Vr_s,VF1_s))
print('Accuary:%.2f\tPrecision:%.2f\tRecall:%.2f\tF1:%.2f\n'%(Ta_s,Tp_s,Tr_s,TF1_s))
shredhold,a,p,r,F1 = Find_shredhold(np.array(torch.sum(torch.cosine_similarity(train_emb[:,:,:LM.dim],train_emb[:,:,LM.dim:],dim=-1),dim=-1)),TrainSet.SR_List)
print(shredhold)
Va,Vp,Vr,VF1 = Evaluatation(np.array(torch.sum(torch.cosine_similarity(valid_emb[:,:,:LM.dim],valid_emb[:,:,LM.dim:],dim=-1),dim=-1)),ValidSet.SR_List,shredhold)
Ta,Tp,Tr,TF1 = Evaluatation(np.array(torch.sum(torch.cosine_similarity(test_emb[:,:,:LM.dim],test_emb[:,:,LM.dim:],dim=-1),dim=-1)),TestSet.SR_List,shredhold)
print('Accuary:%.2f\tPrecision:%.2f\tRecall:%.2f\tF1:%.2f\n'%(Va,Vp,Vr,VF1))
print('Accuary:%.2f\tPrecision:%.2f\tRecall:%.2f\tF1:%.2f\n'%(Ta,Tp,Tr,TF1 ))

0.80611134
Accuary:67.77	Precision:79.65	Recall:54.96	F1:65.04

Accuary:61.01	Precision:60.59	Recall:63.31	F1:61.92

2.105534
Accuary:64.86	Precision:78.60	Recall:48.91	F1:60.30

Accuary:63.39	Precision:63.33	Recall:63.87	F1:63.60



In [ ]:
res_path = r'./result/'
file_path = res_path + encoder +'_'+ pooling + '_unsupvised.txt'
with open(file_path,'w') as f:
        f.write('Multi-Grain\n')
        f.write('Train:\n')
        f.write('Shredhold:%.2f\n'%(shredhold))
        f.write('Accuary:%.2f\tPrecision:%.2f\tRecall:%.2f\tF1:%.2f\n'%(a,p,r,F1))
        f.write('Valid:')
        f.write('Accuary:%.2f\tPrecision:%.2f\tRecall:%.2f\tF1:%.2f\n'%(Va,Vp,Vr,VF1))
        f.write('Test:')
        f.write('Accuary:%.2f\tPrecision:%.2f\tRecall:%.2f\tF1:%.2f\n'%(Ta,Tp,Tr,TF1))
        f.write('Span\n')
        f.write('Train:\n')
        f.write('Shredhold:%.2f\n'%(shredhold_span))
        f.write('Accuary:%.2f\tPrecision:%.2f\tRecall:%.2f\tF1:%.2f\n'%(a_s,p_s,r_s,F1_s))       
        f.write('Valid:')
        f.write('Accuary:%.2f\tPrecision:%.2f\tRecall:%.2f\tF1:%.2f\n'%(Va_s,Vp_s,Vr_s,VF1_s))
        f.write('Test:')
        f.write('Accuary:%.2f\tPrecision:%.2f\tRecall:%.2f\tF1:%.2f\n'%(Ta_s,Tp_s,Tr_s,TF1_s ))
f.close()

**Random Foreast**

In [ ]:
import torch
from torch.nn import functional as F
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import numpy as np  

class RandomForest():
    """Random Forest classifier. Uses a collection of classification trees that
    trains on random subsets of the data using a random subsets of the features.
    Parameters:
    -----------
    n_estimators: int
        The number of classification trees that are used.
    max_features: int
        The maximum number of features that the classification trees are allowed to
        use.
    min_samples_split: int
        The minimum number of samples needed to make a split when building a tree.
    min_gain: float
        The minimum impurity required to split the tree further.
    max_depth: int
        The maximum depth of a tree.
    """

    def __init__(self, n_estimators=100, min_samples_split=2, min_gain=1,
                 max_depth=None, max_features=None):

        self.n_estimators = n_estimators #树的数量
        self.min_samples_split = min_samples_split #每棵树中最小的分割数，比如 min_samples_split = 2表示树切到还剩下两个数据集时就停止
        self.min_gain = min_gain   #每棵树切到小于min_gain后停止
        self.max_depth = max_depth  #每棵树的最大层数
        self.max_features = max_features #每棵树选用数据集中的最大的特征数

        self.trees = []
        # 建立森林(bulid forest)
        for _ in range(self.n_estimators):
            tree = DecisionTreeClassifier(min_samples_split=self.min_samples_split, min_samples_leaf=self.min_gain,
                                      max_depth=self.max_depth)
            self.trees.append(tree)

    def fit(self, X, Y):
        # 训练，每棵树使用随机的数据集(bootstrap)和随机的特征
        # every tree use random data set(bootstrap) and random feature
        sub_sets = self.get_bootstrap_data(X, Y)
        n_features = X.shape[1]

        if self.max_features == None:
            self.max_features = int(np.sqrt(n_features))
        for i in range(self.n_estimators):
            # 生成随机的特征
            # get random feature
            sub_X, sub_Y = sub_sets[i]
            idx = np.random.choice(n_features, self.max_features, replace=True)
            sub_X = sub_X[:, idx]
            self.trees[i].fit(sub_X, sub_Y)
            self.trees[i].feature_indices= idx
            print("tree", i, "fit complete")

    def predict(self, X):
        y_preds = []
        for i in range(self.n_estimators):
            idx = self.trees[i].feature_indices
            sub_X = X[:, idx]
            y_pre = self.trees[i].predict(sub_X)
            y_preds.append(y_pre)
        y_preds = np.array(y_preds).T
        y_pred = []
        for y_p in y_preds:
            # np.bincount()可以统计每个索引出现的次数
            # np.argmax()可以返回数组中最大值的索引
            # cheak np.bincount() and np.argmax() in numpy Docs
            y_pred.append(np.bincount(y_p.astype('int')).argmax())
        return y_pred

    def get_bootstrap_data(self, X, Y):

        # 通过bootstrap的方式获得n_estimators组数据
        # get int(n_estimators) datas by bootstrap

        m = X.shape[0] #行数
        Y = Y.reshape(m, 1)

        # 合并X和Y，方便bootstrap (conbine X and Y)
        X_Y = np.hstack((X, Y)) #np.vstack():在竖直方向上堆叠/np.hstack():在水平方向上平铺
        np.random.shuffle(X_Y) #随机打乱

        data_sets = []
        for _ in range(self.n_estimators):
            idm = np.random.choice(m, m, replace=True) #在range(m)中,有重复的选取 m个数字
            bootstrap_X_Y = X_Y[idm, :]
            bootstrap_X = bootstrap_X_Y[:, :-1]
            bootstrap_Y = bootstrap_X_Y[:, -1:]
            data_sets.append([bootstrap_X, bootstrap_Y])
        return data_sets


if __name__ == '__main__':
    clf = RandomForest(n_estimators=20)
    # '''L2 span'''
    # clf.fit(np.array((train_emb[:,0,:LM.dim] - train_emb[:,0,LM.dim:])**2), np.array(TrainSet.SR_List))
    # test_y_pred = clf.predict(np.array((test_emb[:,0,:LM.dim] - test_emb[:,0,LM.dim:])**2))
    # valid_y_pred = clf.predict(np.array((valid_emb[:,0,:LM.dim] - valid_emb[:,0,LM.dim:])**2))
    # '''L2 multi-grain'''
    # clf.fit(np.array(torch.sum((train_emb[:,:,:LM.dim] - train_emb[:,:,LM.dim:])**2,dim=1)), np.array(TrainSet.SR_List))
    # test_y_pred = clf.predict(np.array(torch.sum((test_emb[:,:,:LM.dim] - test_emb[:,:,LM.dim:])**2,dim=1)))
    # valid_y_pred = clf.predict(np.array(torch.sum((valid_emb[:,:,:LM.dim] - valid_emb[:,:,LM.dim:])**2,dim=1)))
    '''L1 span'''
    clf.fit(np.array(torch.abs(train_emb[:,0,:LM.dim] - train_emb[:,0,LM.dim:])), np.array(TrainSet.SR_List))
    test_y_pred = clf.predict(np.array(torch.abs(test_emb[:,0,:LM.dim] - test_emb[:,0,LM.dim:])))
    valid_y_pred = clf.predict(np.array(torch.abs(valid_emb[:,0,:LM.dim] - valid_emb[:,0,LM.dim:])))
    # '''cat'''
    # clf.fit(np.array(train_emb[:,0,]), np.array(TrainSet.SR_List))
    # test_y_pred = clf.predict(np.array(test_emb[:,0,]))
    # valid_y_pred = clf.predict(np.array(valid_emb[:,0,]))


In [ ]:
def Evaluate(pred,truth,shredhold = None):
    TP,FP,TN,FN = 0,0,0,0
    Accuary,Precision,Recall,F1 = 0,0,0,0
    for i in range(len(truth)):
        if truth[i]:
            if not shredhold:
                if pred[i] == 1:
                    TP+=1
                else: FN+=1
            else:
                if pred[i] >= shredhold:
                    TP+=1
                else: FN+=1
        else:
            if not shredhold:
                if pred[i] == 1:
                    FP+=1
                else: TN+=1
            else:
                if pred[i] < shredhold:
                    TN+=1
                else: FP+=1
    Accuary = (TP+TN)/(TP+FP+TN+FN)*100
    Precision = TP/(TP+FP)*100
    Recall = TP/(TP+FN)*100
    F1 = 2*(Precision*Recall)/(Precision+Recall)
    return Accuary,Precision,Recall,F1

Va,Vp,Vr,VF1 = Evaluatation(valid_y_pred,ValidSet.SR_List)
Ta,Tp,Tr,TF1 = Evaluatation(test_y_pred,TestSet.SR_List)
print('Accuary:\t%.2f\tPrecision:\t%.2f\tRecall:\t%.2f\tF1:\t%.2f\n'%(Va,Vp,Vr,VF1))
print('Accuary:\t%.2f\tPrecision:\t%.2f\tRecall:\t%.2f\tF1:\t%.2f\n'%(Ta,Tp,Tr,TF1))

res_path = r'./result/RF/'
file_path = res_path + '100_L1.txt'
with open(file_path,'w') as f:
        f.write('Multi-Grain\n')
        f.write('Valid:\n')
        f.write('Accuary:\t%.2f\tPrecision:\t%.2f\tRecall:\t%.2f\tF1:\t%.2f\n'%(Va,Vp,Vr,VF1))
        f.write('Test:\n')
        f.write('Accuary:\t%.2f\tPrecision:\t%.2f\tRecall:\t%.2f\tF1:\t%.2f\n'%(Ta,Tp,Tr,TF1))

In [ ]:
# from sklearn.externals import joblib
import joblib
# import cPickle
# with open(r'./models/RF_L2_MGR.m','wb') as f:
    # cPickle.dump(clf, f)
joblib.dump(clf,r'./models/RF/100_scibert_cat.m')

In [28]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
test_pred = np.array(torch.cosine_similarity(test_emb[:,:,:LM.dim],test_emb[:,:,LM.dim:],dim=-1)[:,0])
positive,negative = [],[]
for i in range(len(TestSet.SR_List)):
    if TestSet.SR_List[i]:
        positive.append(test_pred[i])
    else:
        negative.append(test_pred[i])
# plt.subplot(1,2,1)
# plt.hist(positive,color='red',label='positive')
# plt.subplot(1,2,2)
# plt.hist(negative,color='blue',label='nengative')
# plt.savefig('./unsupervised.jpg')

In [29]:
path = r'./pred/scibert_mean_dssm_100_50_200_202302172028.txt'
d_positive,d_negative = [],[]
with open(path,'r') as f:
    lines = f.readlines()
f.close()
for line in lines:
    if 'Right' in line or 'Wrong' in line:
        if line.endswith('1\n'):
            d_positive.append(float(line.split('\t')[0]))
        elif line.endswith('0\n'):
            d_negative.append(float(line.split('\t')[0]))
        else:pass
    else:pass
# plt.subplot(2,1,1)  
# n, bins, patches = plt.hist(d_positive,color='red')
# plt.subplot(2,1,2)
# n, bins, patches = plt.hist(d_negative,color='blue')
# plt.savefig('./DSSM.jpg')

In [ ]:
plt.subplot(2,2,1)
plt.title('Without DSSM')
plt.hist(positive,color='red')
plt.subplot(2,2,2)
plt.title('DSSM')
plt.hist(d_positive,color='red')
plt.subplot(2,2,3)
plt.hist(negative,color='blue')
plt.subplot(2,2,4)
plt.hist(d_negative,color='blue')
plt.savefig('./DSSM_RES.jpg')

In [ ]:
file_path = r'./DSSM_AB.csv'
with open(file_path,'w') as f:
    f.write('Without DSSM\n')
    f.write('P:\t'+'\t'.join([str(i) for i in positive])+'\n')
    f.write('N:\t'+'\t'.join([str(i) for i in negative])+'\n')
    f.write('DSSM\n')
    f.write('P:\t'+'\t'.join([str(i) for i in d_positive])+'\n')
    f.write('N:\t'+'\t'.join([str(i) for i in d_negative])+'\n')


In [49]:
fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2)

a = [np.array(positive),np.array(negative)]
ax0.hist(a,bins=10,label=['positive','negative'])
ax0.legend(prop={'size': 10})
plt.subplot(1,2,1)
plt.suptitle('(a)')
# ax0.set_suptitle('(a)',loc='left')


b = [np.array(d_positive),np.array(d_negative)]
ax1.hist(b,bins=10,label=['positive','negative'])
ax1.legend(prop={'size': 10})
plt.subplot(1,2,2)
plt.suptitle('(b)')
# ax1.set_suptitle('(b)',loc='left')

plt.savefig('./DSSM_RES05.jpg',dpi=300)

/home/jsj201-7/mount1/anaconda3/envs/mimo/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).size
/home/jsj201-7/mount1/anaconda3/envs/mimo/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:2082: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.atleast_1d(X.T if isinstance(X, np.ndarray) else np.asarray(X))


AttributeError: 'AxesSubplot' object has no attribute 'set_suptitle'